#  > EDA - PROJETO HIDROMETROS

### **Solução proposta**
#### _Modelo que defina o momento ideal de troca do hidrometro levando em consideração a perda de eficiência e a possível mudança de faixa._

### **Variáveis que influenciam a vida útil**
1. ####  Pressão da rede
2. #### Volume medido durante a vida
3. #### Marca
4. #### Capacidade 
5. #### Qualidade da agua da rede
6. #### Recondicionamento

### **Datasets**
1. #### Hidrometros (PRAX).
2. #### Analises de conformidade (PEA) (Xls)

### ** Observações **
- #### Normativa do inmetro recomenda aferições/regulagens no periodo de 5 anos.
- #### Hidrometros com mais de 10 anos de idade são descartados automáticamente.
- #### Análise de perda aplicando perfil de consumo baseado na ABNT NBR 15538 e erros de medição verificados através do serviço 070.


```sql
-- MEDICOES 2018
SELECT 
	MED.MED_NUM_MEDIDOR, 
	MED.MED_NUM_SEQ_MEDIDOR,
	MED.MED_DAT_CADASTRO_MEDIDOR,
	MED.MAR_SEQ_MARCA,
	MED.ILM_SEQ_ITEM_LOTE_MEDIDOR,
	MED.MED_STA_MEDIDOR,
	MED.ESM_SEQ_ESPECIFICACAO_MEDIDOR,
	INS.INS_DAT_INSTALACAO_MEDIDOR,
	INS.INS_DAT_RETIRADA_MEDIDOR,
	MDC.MED_NUM_LEITURA_CAMPO_ANT,
	MDC.MED_NUM_LEITURA_PROC_ANT,
	MDC.MED_NUM_LEITURA_CAMPO_ATUAL,
	MDC.MED_NUM_LEITURA_PROC_ATUAL,
	MDC.MED_DAT_LEITURA,
	MDC.OCL_SEQ_OCORRENCIA_CAMPO,
	LIG.IMO_COD_INSCRICAO,
	LIG.PRO_SEQ_PRODUTO
FROM MED_MEDICAO MDC
JOIN INS_INSTALACAO  INS 	USING (INS_SEQ_INSTALACAO)
JOIN MED_MEDIDOR MED 		USING (MED_SEQ_MEDIDOR)
JOIN LIG_LIGACAO LIG 		USING(LIG_SEQ_LIGACAO)
WHERE MED_CPT_MEDICAO BETWEEN 201801 AND 201812;
```

>Um medidor (med_medidor) esta associado a uma ou mais ligacoes (lig_ligacao)  das quais possuem uma ou mais instalações (ins_instalacao) que por sua vez possuem uma ou mais medicoes(med_medicao)

## FUNÇÕES

## IMPORTS / CONFIGS

In [1]:
'''%whos DataFrame'''
import warnings ; warnings.filterwarnings("ignore", category=RuntimeWarning) 
import sys ; sys.path.append('../scripts')
import pdmunging as pdm
import eda_hidrometros as eda

In [2]:
from pandas import merge as pdmerge
from pandas.io.parsers import read_csv
import operator
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql


# only on explore
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
plt.style.use('fivethirtyeight')
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## LOAD

In [30]:
df_substituicoes = read_csv('../data/dataset_substituicoes_2017_AMOSTRA1.csv', 
                            parse_dates=['REL_DAT_SUBSTITUICAO', 'REL_DAT_INS_ANTERIOR', 'REL_DAT_INSTALACAO'])
df_servico_70 = read_csv('../data/serv70_2018.csv',decimal=',', thousands='.',\
                    names=['UN','dataVerificacao','IDHidrometro','dataInstalacao', 'Qn', \
                           'Qt', 'Qmin', 'status', 'tipoReprovacao', 'Impedimento'], skiprows=1,\
                        parse_dates=['dataVerificacao', 'dataInstalacao'])
df_historico_hidrometros = read_csv('../data/tipo_lote_hidrometro.csv',parse_dates=['LOM_DAT_FECHAMENTO_LOTE'])
df_elevacoes = read_csv('../data/elevacao.csv')

In [ ]:
# exportar dados para coletar medições dos hidrometros do serviço 70
df_servico_70['dataVerificacaoINI'] = df_servico_70['dataVerificacao'] - pd.DateOffset(months=25)
df_servico_70[['dataVerificacao', 'dataVerificacaoINI', 'IDHidrometro']]\
    .to_csv('../data/PeriodoPorHidrometro.csv', index=False, date_format='%Y%m')

In [31]:
# Leitura do dataset gerado no etl
colunas_leitura = ['MED_NUM_SEQ_MEDIDOR','MED_NUM_MEDIDOR','MED_DAT_CADASTRO_MEDIDOR','MAR_SEQ_MARCA',\
           'ILM_SEQ_ITEM_LOTE_MEDIDOR','MED_STA_MEDIDOR','ESM_SEQ_ESPECIFICACAO_MEDIDOR',\
           'INS_DAT_INSTALACAO_MEDIDOR','INS_DAT_RETIRADA_MEDIDOR','MED_NUM_LEITURA_CAMPO_ANT',\
           'MED_NUM_LEITURA_PROC_ANT','MED_NUM_LEITURA_CAMPO_ATUAL','MED_NUM_LEITURA_PROC_ATUAL',\
           'MED_DAT_LEITURA','OCL_SEQ_OCORRENCIA_CAMPO','IMO_COD_INSCRICAO','PRO_SEQ_PRODUTO', 'QTD_INSTALACOES']

df_leitura = read_csv('../data/DATASET_ETL_MEDICOES.csv', names=colunas_leitura, low_memory=False, \
                      parse_dates=['MED_DAT_CADASTRO_MEDIDOR','INS_DAT_INSTALACAO_MEDIDOR',\
                                   'INS_DAT_RETIRADA_MEDIDOR','MED_DAT_LEITURA'], \
                      decimal='.', thousands=',')

In [ ]:
# exportar dados para coletar dados dos imoveis
df_leitura['IMO_COD_INSCRICAO'].drop_duplicates(keep='first').to_csv('../data/ImoveisNoPeriodo.csv', index=False)

In [32]:
# Leitura do dataset gerado no etl
colunas_imovel = ['IMO_COD_INSCRICAO','IMO_QTD_PONTOS_UTILIZACAO','PIM_SEQ_PADRAO_IMOVEL',\
                  'IMO_QTD_AREA_COBERTA','TIC_SEQ_TIP_CONSTRUCAO','UAD_COD_UNIDADE_ADMINISTRATIVA',\
                  'SET_TIP_SETOR']
df_imovel = read_csv('../data/DATASET_ETL_IMOVEIS.csv', names=colunas_imovel)

## PIPELINE

In [33]:
df_historico_hidrometros.pipe(eda.TipoHidrometroMap, col='LOM_COD_TIPO_LOTE')

df_servico_70.pipe(eda.RemoveEspaco, col='IDHidrometro', value='')\
             .pipe(eda.CalculaPerda,  cols=['Qt', 'Qmin', 'Qn'], factors=[0.7403, 0.1838, 0.0759], colName='Perda' )

df_servico_70 = df_servico_70[df_servico_70['Impedimento'].isna()]
print('-')

-


In [34]:
df_leitura.pipe(eda.RelacionaColuna, colName='IDHidrometro', col1=df_leitura['MED_NUM_MEDIDOR'].astype(object), \
              col2=df_leitura['MED_NUM_SEQ_MEDIDOR'].astype(str), operador=operator.concat)\
        .pipe(eda.AnoMedidorLacre, col='MED_NUM_MEDIDOR', colName='MED_ANO_MEDIDOR')\
        .pipe(eda.RelacionaColuna, colName='ConsumoProcessado', col1=df_leitura.MED_NUM_LEITURA_PROC_ATUAL, \
              col2=df_leitura.MED_NUM_LEITURA_PROC_ANT, operador=operator.sub)\
        .pipe(eda.RelacionaColuna, colName='ConsumoMedido', col1=df_leitura.MED_NUM_LEITURA_CAMPO_ATUAL, \
              col2=df_leitura.MED_NUM_LEITURA_CAMPO_ANT, operador=operator.sub)\
        .pipe(eda.TrataNegativos, col='ConsumoProcessado', use=np.nan)\
        .pipe(eda.ColunaTransformada,  colName='MedianaProcessado', groupby='IDHidrometro', \
              col='ConsumoProcessado', operation='median')\
        .pipe(eda.ColunaTransformada,  colName='QtdMedicoes', groupby='MED_NUM_SEQ_MEDIDOR', \
              col='IMO_COD_INSCRICAO', operation='count')\
        .pipe(eda.ColunaTransformada,  colName='MediaProcessado', groupby='IDHidrometro', \
              col='ConsumoProcessado', operation='mean')\
        .pipe(eda.ColunaTransformada,  colName='DesvioPadraoProcessado', groupby='IDHidrometro', \
              col='ConsumoProcessado', operation='std')\
        .pipe(eda.RelacionaColuna, colName='CVProcessado', col1=df_leitura.DesvioPadraoProcessado, \
              col2=df_leitura.MediaProcessado, operador=operator.truediv)\
        .pipe(eda.AplicaFuncao, colName='isOcorrencia', col='OCL_SEQ_OCORRENCIA_CAMPO',\
              funcao = lambda x: 1 if x > 0 else 0 )\
        .pipe(eda.ColunaTransformada,  colName='QtdOcorrencias', groupby='IDHidrometro', \
              col='isOcorrencia', operation='sum')\
        .pipe(eda.ColunaTransformada,  colName='variacaoTendencia', groupby='IDHidrometro', \
              col='ConsumoProcessado', operation=eda.trendserie)
print('-')

-


In [35]:
colunas_analise = ['IDHidrometro','MED_ANO_MEDIDOR', 'IMO_COD_INSCRICAO', 'MAR_SEQ_MARCA', \
                   'ESM_SEQ_ESPECIFICACAO_MEDIDOR','MedianaProcessado', 'MediaProcessado', \
                   'DesvioPadraoProcessado','CVProcessado', 'QtdOcorrencias', 'INS_DAT_INSTALACAO_MEDIDOR', 'QtdMedicoes', 'QTD_INSTALACOES', 'variacaoTendencia']
df_leitura = df_leitura[colunas_analise].drop_duplicates(keep='first')

In [36]:
df = pdmerge(df_servico_70, df_leitura, how='left', \
             on="IDHidrometro", suffixes=('_srv70', '_medicao'))
df = pdmerge(df, df_imovel, how='left', \
             on="IMO_COD_INSCRICAO", suffixes=('_df', '_imovel'))

In [37]:
df.pipe(eda.TrataNA, col='dataInstalacao', use=df['INS_DAT_INSTALACAO_MEDIDOR'])\
  .pipe(eda.Recorta, col='status' , start_position=0, end_position=2)\
  .pipe(eda.AplicaFuncao, colName='anoInstalacao', col='dataInstalacao',\
              funcao = lambda x: pd.Timestamp(x).year )\
  .pipe(eda.AplicaFuncao, colName='anoRetirada', col='dataVerificacao',\
              funcao = lambda x: pd.Timestamp(x).year )\
  .pipe(eda.RelacionaColuna, colName='vidaInstalacao', col1=df['anoRetirada'], \
              col2=df['anoInstalacao'], operador=operator.sub)\
  .pipe(eda.RelacionaColuna, colName='vidaHidrometro', col1=df['anoRetirada'], \
              col2=df['MED_ANO_MEDIDOR'], operador=operator.sub)\
  .pipe(eda.setOutlier, 'CVProcessado', 'outCV')\
  .pipe(eda.setOutlier, 'variacaoTendencia', 'outVarTend')
print('-')

-


In [38]:
remover_do_df = ['UN', 'dataVerificacao', 'IDHidrometro', 'dataInstalacao', 'Qn', 'Qt', 'Qmin', \
                 'tipoReprovacao', 'Impedimento', 'Perda', 'MED_ANO_MEDIDOR', 'IMO_COD_INSCRICAO', \
                 'INS_DAT_INSTALACAO_MEDIDOR','SET_TIP_SETOR', 'anoInstalacao', 'anoRetirada', \
                'TIC_SEQ_TIP_CONSTRUCAO', 'IMO_QTD_AREA_COBERTA', 'DesvioPadraoProcessado', \
                 'MediaProcessado', 'MedianaProcessado' ]
df.drop(columns=remover_do_df, inplace=True)

aplicar_onehotencoding = ['MAR_SEQ_MARCA','ESM_SEQ_ESPECIFICACAO_MEDIDOR', 'PIM_SEQ_PADRAO_IMOVEL', \
                         'UAD_COD_UNIDADE_ADMINISTRATIVA']
for categoria in aplicar_onehotencoding:
    df = pdm.set_onehotencoding(df, categoria, categoria)

df.dropna(inplace=True)

selected_to_del = df[(df.vidaInstalacao < 5) & (df.status == 'RP') & (df.CVProcessado < 1)& (df.vidaHidrometro < 10)].index
df.drop(selected_to_del, inplace=True)
df.drop(df[df.QtdMedicoes>14].index, inplace=True)
df.drop(df[df['outCV'] ==1].index, inplace=True)
df.drop(df[df['outVarTend'] ==1].index, inplace=True)

status_map = {'RP': 0, 'AP': 1}
df['status'] = df['status'].map(status_map)
#df['status'].value_counts()
#df[df.QtdMedicoes>14].info()

In [39]:
df.to_csv('../data/train_hidrometros.csv', index=False)

```python
#code for profilling of df
import matplotlib.pyplot as plt
import pandas_profiling as pf
df_to_proffiling = df[['CVProcessado', 'QtdOcorrencias','QtdMedicoes','QTD_INSTALACOES',\
                       'IMO_QTD_PONTOS_UTILIZACAO', 'vidaInstalacao', 'vidaHidrometro']]
pf.ProfileReport(df_to_proffiling)
```